# 被动网络测量

网络运营者会查看不同类型的网络流量数据来了解其网络的属性。有些网络数据可以在设备（例如路由器、交换机）转发实时流量时直接收集。收集这些数据不会影响网络行为，因此称为“被动”（相比之下，“主动”测量会发起探测流量）。

在本次作业中，你将分析两类被动网络测量数据——流量量（traffic volumes）和 BGP 路由。

本 notebook 分为若干部分。每一部分包含带有 TODO 的章节，需要你完成。

**请在下面单元格中填写你的姓名和 netID：**

**姓名：**

**NetId：**

## 背景

### 用 IPFIX 进行流量测量

大多数网络中的路由器使用 IPFIX 协议来收集流量测量数据。[NetFlow]（一种由 Cisco 定义的专有形式的 IPFIX）在网络研究社区中很有名，因为 Cisco 为许多大型网络提供路由器。

在本部分中，你将分析从连接普林斯顿校园网络到互联网的路由器捕获的一段 NetFlow 记录。该作业会要求你执行与网络运营者类似的分析——例如：找出校园网络上最受欢迎的流量端点、最常用的 Web 应用等等。（可以想象，当我们考虑安全性时，分析这些基线会非常有用。）

流记录位于此文件夹下的 `netflow.csv` 文件中。为简化分析，我们保证普林斯顿校园网络的 IP 地址以 128.112 开头，并且已对低 16 位进行匿名化以保护校园用户的隐私（即对用户做了混淆）。为进一步简化任务，我们将这些记录解析成 CSV（逗号分隔）格式，并在文件第一行列出了字段名。（在真实环境中，路由器以二进制格式导出 IPFIX 记录。）

### 用 BGP 路由表测量域间路由

为了帮助网络运营者了解 Internet 路由的状态，许多路由器能够定期将 BGP 路由表“导出”成静态文件。这些路由表包含了每个 IP 前缀、路由器为该前缀学习到的所有 BGP 路由，以及路由器最终选择的“最佳”路由。分析 BGP 路由表可以提供有关到不同 IP 前缀的流量将被送往何处的信息。

在本作业中，我们为你提供了来自一个名为 RouteViews 的项目的路由表转储（dump）。你可以访问该站点了解它们收集的路由表。`telnet route-views2.routeviews.org` 也会给你一个命令行提示符，让你在 routeviews 项目的一台真实 BGP 路由器上查看实时路由表（如果你想玩的话）。http://routeviews.org/bgpdata/ 提供周期性的二进制路由表转储和更新日志供参与路由器使用。同样地，为了本作业，我们将二进制路由表转储解析为更易于分析的格式，数据位于 `bgp_rib.csv` 文件中。

### 使用 map() 和 reduce() 的函数式数据分析

本作业中的若干数据分析步骤使用“MapReduce”编程模型。MapReduce 源自函数式编程语言，它涉及两个函数（称为 `map()` 和 `reduce()` —— 惊不惊喜！）来对可迭代的数据（比如列表、数组等）应用函数。

##### map()

通用的 `map()` 函数有两个参数：另一个函数（这个函数本身接受一个参数）和一个可迭代对象。`map()` 会把该函数应用到可迭代对象的每一个元素上。请参阅 Python 内置 `map()` 的文档了解更多：https://docs.python.org/2/library/functions.html#map。下面的示例使用 `map()` 将列表的每个元素加 3。

In [ ]:
some_numbers = [1,2,3]
three_more = map(lambda x: x+3, some_numbers)
print three_more

`map()` 常常与匿名函数（上例中的 `lambda`）一起使用，但也可以与普通函数一样使用：

In [ ]:
def add3(i):
    return i+3
 
some_numbers = [1,2,3]
three_more = map(add3, some_numbers)
print three_more

注意：真实实现的 `map()` 允许被映射的函数接受多个参数或包含闭包中的更多信息，但这些在本作业中不会必要。

##### reduce()

通用的 `reduce()` 函数接受另一个函数（该函数本身接受 *两个* 值）、一个可迭代对象，以及一个可选的初始化值。该函数先对可迭代对象的前两个元素（或第一个元素和初始化值）应用一次以得到返回值。然后函数再把该返回值与下一个元素作为参数继续调用，依此类推，直到迭代对象被归约为单个返回值。这样 `reduce()` 就能对整个可迭代对象计算汇总值。参见 Python 内置 `reduce()` 的文档了解更多：https://docs.python.org/2/library/functions.html#reduce。下面的例子使用 `reduce()` 来统计列表中数字 4 的个数：

In [ ]:
def count_4s(count, i):
    # 参数的顺序很重要。
    #     第一个参数是累积的值
    #     第二个参数是来自可迭代对象的下一个值
    if i == 4:
        return count + 1
    else:
        return count

some_numbers = [1,4,0,1,4]
num_fours = reduce(count_4s, some_numbers, 0) # 0 是初始化值
print num_fours

同样地，真实实现的 `reduce()` 允许归约函数接受多个参数或包含闭包中的更多信息，但在本作业中不需要这些扩展。

[MapReduce](https://en.wikipedia.org/wiki/MapReduce) 之所以流行，是因为它允许对大数据集的分析任务进行并行化。虽然存在许多开源和专有的 MapReduce 风格数据处理库（通常在表达可迭代数据集和在多台机器间分配任务的方式上有所不同），但它们都涉及像本作业中使用的 `map()` 和 `reduce()` 函数。如果你对函数式编程用于数据分析感兴趣，可以考虑选修 COS 326。

## A 部分：IPFIX 数据

### 解析 IPFIX 数据
`netflow.csv` 文件包含来自普林斯顿校园网络边界路由器的预处理 netflow 数据。该数据是“无采样（unsampled）”的，也就是说它统计了边界路由器任一接口上转发的每个数据包的流量汇总。我们使用 `nfdump` 工具处理原始 NetFlow 数据并生成了这些记录。`netflow.csv`（除第一行标题外）的每一行记录一个流（flow）的如下信息：

```
首次出现日期, 首次出现时间 (m:s), 最后出现日期, 最后出现时间 (m:s), 持续时间 (s), 协议, 
源 IP 地址, 源端口, 目的 IP 地址, 目的端口, 包数, 字节数, 标志, 输入接口, 输出接口		
```

为分析这些数据，我们首先需要把它读入一个 Python 数据结构。下面的代码使用内置的 `csv` 库把 `netflow.csv` 读取为一个字典列表（list of dicts）。有关 `csv` 库的文档，请参阅：https://docs.python.org/2/library/csv.html

In [ ]:
import csv

with open('netflow.csv', 'r') as netflow_file:
    netflow_reader = csv.DictReader(netflow_file)
    netflow_data = list(netflow_reader)
    
print "Number of flow records: {}".format(len(netflow_data))
print
print "Sample flow record: {}".format(netflow_data[0])

### 分析 IPFIX 数据

下面的各节针对使用 netflow 数据回答特定问题展开。这些问题对真实的网络运营者很重要，并且可能显示普林斯顿网络使用互联网的某些惊人事实。

#### 哪些 IP 地址是普林斯顿网络用户最常访问的？

为了回答这个问题，我们需要决定如何衡量 IP 地址的“受欢迎程度”。可以使用总流量（bytes）作为度量，也可以使用到某个 IP 地址的流（flow）总数作为度量。网络运营者实际会同时使用这两类度量（以及其他指标），我们在下面也会同时使用它们。

*步骤 1：按流（number of flows）统计最受欢迎的 IP 地址*

请完成下面的代码以生成一个 Python 字典 `ips_by_flows`，其统计 `netflow_data` 中每个外部（非 128.112.*.*）目的 IP 地址的流数量。字典的键应为 IP 地址，值为整数流计数。

首先完成 `count_by_flows()` 函数，它应接收当前的字典结果并根据 `current_flow` 更新它。如果对数据类型有疑问，可以用打印语句检查变量。

你可能想写一个辅助函数来判断 IP 地址是否以 128.112 开头。可参考字符串方法文档：https://docs.python.org/2/library/stdtypes.html#string-methods。

接着你需要使用 `reduce()` 函数来构建字典结果。提示：对 `reduce()` 的初始化值应使用 `defaultdict(lambda: 0)`。`defaultdict()` 会创建一个在键不存在时返回默认值（这里为 0）的字典，这样你就可以直接递增键的值而不需要先检查键是否存在（如果使用普通的 `{}`，首次访问不存在的键时会抛出 KeyError）。

下面的代码会打印并绘制最受欢迎的 IP 地址。`check_ips_by_flows()` 函数会把你得出的前 15 个最受欢迎 IP 的 md5 哈希与正确答案比较，以告知你是否正确或需要继续调试。

In [ ]:
%matplotlib inline
from collections import defaultdict
from plotting import plot_flows
from testing import check_ips_by_flows

# TODO: complete count_by_flows function
def count_by_flows(counts, current_flow):
    # counts 是当前的字典结果
    # current_flow 是正在处理的流记录
    pass
      

# TODO: use reduce() function to apply count_by_flows to netflow_data and assign the result to ips_by_flows


# print the top 5 IP addresses by number of flows 
sorted_ips_by_flows = sorted(ips_by_flows.items(), reverse=True, key=lambda x: x[1])
print "Most popular IP addresses by number of flows: {}\n".format(sorted_ips_by_flows[0:5])

# check the results
check_ips_by_flows(sorted_ips_by_flows[0:15])

# plot the results
plot_flows(sorted_ips_by_flows)

*步骤 2：按总流量（total volume）统计最受欢迎的 IP 地址*

请完成以下代码以生成字典 `ips_by_volume`，它统计每个外部（非普林斯顿）目的 IP 地址的总字节数。字典的键为 IP 地址，值为整数字节数。请注意 `netflow_data` 中的数值通常为字符串类型，需要转换为 int 或 float 才能进行算术运算。

In [ ]:
%matplotlib inline
from plotting import plot_volumes
from testing import check_ips_by_volume

# TODO: complete count_by_volume function
def count_by_volume(counts, current_flow):
    # counts 是当前的字典结果
    # current_flow 是正在处理的流记录
    pass

        
# TODO: use reduce() function to apply count_by_volume to netflow_data and assign the result to ips_by_volume


# print the top 5 IP addresses by volume
sorted_ips_by_volume = sorted(ips_by_volume.items(), reverse=True, key=lambda x: x[1])
print "Most popular IP addresses by volume: {}\n".format(sorted_ips_by_volume[0:5])

# check the results
check_ips_by_volume(sorted_ips_by_volume[0:15])

# plot the results
plot_volumes(sorted_ips_by_volume)

#### 普林斯顿网络用户最常使用的应用（按协议）是什么？

你认为在普林斯顿网络上哪些应用协议最常见？Web 流量（HTTP & SSL）？安全远程连接（SSH）？邮件（SMTP、IMAP、POP3）？在实践中，网络运营者可能想识别常用应用以做容量规划或更改网络配置（例如，不同应用的流量通过不同链路），以防高流量的应用（如视频流）影响关键低流量应用的性能。

你可以通过找出 netflow 数据中最常见的端口来回答这些问题。许多应用协议使用“知名端口”。例如：HTTP 使用端口 80，SSL 使用端口 443，SSH 使用端口 22，SMTP 使用端口 25。

我们同样使用“流数量”和“总流量”两种度量来衡量端口的“受欢迎程度”。

请完成下面的代码以从 netflow_data 中创建 `ports_by_flows` 和 `ports_by_volume` 字典，使用与前面对 IP 的统计相同的策略。

包含所有目的端口和源端口中 **小于 1024** 的端口（小于 1024 的端口是“知名端口”，便于映射到应用）。

In [ ]:
%matplotlib inline
from plotting import plot_ports
from testing import check_ports_by_flows, check_ports_by_volume

# TODO: create ports_by_flows and ports_by_volume dicts from netflow_data


# Print the most popular ports and check the results
sorted_ports_by_flows = sorted(ports_by_flows.items(), reverse=True, key=lambda x: x[1])
sorted_ports_by_volume = sorted(ports_by_volume.items(), reverse=True, key=lambda x: x[1])
print "Most popular ports by number of flows: {}".format(sorted_ports_by_flows[0:5])
print "Most popular ports by volume: {}\n".format(sorted_ports_by_volume[0:5])
check_ports_by_flows(sorted_ports_by_flows[0:15])
check_ports_by_volume(sorted_ports_by_volume[0:15])

# plot the results 
plot_ports(sorted_ports_by_flows, sorted_ports_by_volume)

### 问题

请回答下面关于上述分析结果的问题。

#### Q1.
按流数量和按流量，哪些是 5 个最受欢迎的外部（非普林斯顿）IP 地址？

#### A1.
*TODO：在此填写你的答案。*


#### Q2.
在你的 Vagrant 终端中使用 `whois` 命令查询这些 IP（例如 `whois 169.54.233.126`）。选择 Q1 中的两个地址，写出你查到的信息，以及你认为它们为何会如此受欢迎。

#### A2.
*TODO：在此填写你的答案。*


#### Q3.
按流数量和按流量，哪些是 5 个最受欢迎的端口？这些端口对应哪些应用？有关固定端口的维基百科页面：https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers。你也可以在线搜索端口与应用的映射。你对哪些应用最受欢迎感到惊讶吗？

#### A3.
*TODO：在此填写你的答案。*


#### Q4.
你认为为什么 telnet（端口 23）在普林斯顿网络的流量中占比很大？

#### A4.
*TODO：在此填写你的答案。*


#### Q5.
提供的 NetFlow 数据是在大约早上 6:05 到 6:10 的 5 分钟期间捕获的。你认为捕获时间对得到的最受欢迎应用有多大影响？如果改为在你选择的另一个 5 分钟窗口进行采集，你期望看到哪些变化？

#### A5.
*TODO：在此填写你的答案。*


## B 部分：分析 BGP 路由表

到目前为止，我们在 IP 地址和端口级别上查看了流量，但网络运营者可能还想知道哪些其他网络（即自治系统，AS）负责向其网络发送或接收流量。从我们之前关于对等（peering）和互联网商业关系的讲座中，你应该能理解为什么运营者会关心哪些 AS 在向它发送流量！这些信息对分析各种网络攻击（例如拒绝服务攻击的来源）也很有用（我们将在下一次作业中探讨）。

RouteViews 项目允许网络运营者从互联网上不同自治系统的视角获得实时的全局路由信息。RouteViews 服务器充当软件 BGP 路由器，通过 BGP 会话像其他路由器一样学习路由。RouteViews 服务器与普通路由器的主要不同点是它们并不转发真实的互联网流量。

RouteViews 定期以 MRT 的二进制格式记录 BGP 路由表（也称为 RIB）。我们从其中一台服务器收集数据并使用 `bgpdump` 工具将其解析成更可读的输出格式。BGP RIB 条目看起来像下面这样：
```
TIME: 03/07/16 02:00:00
TYPE: TABLE_DUMP_V2/IPV4_UNICAST
PREFIX: 0.0.0.0/0
SEQUENCE: 0
FROM: 185.44.116.1 AS47872
ORIGINATED: 03/06/16 20:27:05
ORIGIN: IGP
ASPATH: 47872 3356
NEXT_HOP: 185.44.116.1
COMMUNITY: 3356:2 3356:514 3356:2087 47872:1 47872:3356

TIME: 03/07/16 02:00:00
TYPE: TABLE_DUMP_V2/IPV4_UNICAST
PREFIX: 0.0.0.0/0
SEQUENCE: 0
FROM: 80.241.176.31 AS20771
ORIGINATED: 03/04/16 10:21:21
ORIGIN: IGP
ASPATH: 20771 1299
NEXT_HOP: 80.241.176.31
```

BGP RIB 可能对一个 IP 前缀有多个条目。

在本作业中，我们对每个 IP 前缀只考虑单个条目。我们把这些解析后的数据写入了 `bgp_rib.csv` 文件。每条记录包含如下字段：
```
TIME, ORIGIN, FROM, SEQUENCE, ASPATH, PREFIX, NEXT_HOP
```

下面的代码把 `bgp_rib.csv` 导入为字典列表（与前面的 netflow 数据处理方式相同）。

In [ ]:
import csv

with open('bgp_rib.csv', 'r') as bgp_file:
    bgp_reader = csv.DictReader(bgp_file,delimiter=";")
    bgp_data = list(bgp_reader)
    
print "Number of BGP RIBs: {}".format(len(bgp_data))
print
print "Sample BGP RIB: {}".format(bgp_data[0])

#### BGP RouteViews 数据中最长的路由（按唯一 AS 数量）是什么？

要回答此问题，你需要根据字段 `ASPATH` 中**唯一** AS 编号的数量对 `bgp_data` 进行排序。

请完成下面的代码，找到最长的 ASPATH 并把它赋给变量 `longest_aspath`。你可以使用 Python 内置的 `sorted()` 函数（https://docs.python.org/2/library/functions.html#sorted）或其它你喜欢的方法。

In [ ]:
from testing import check_longest_aspath

# TODO: Find the longest (by number of *unique* AS numbers) ASPATH and assign it to the 'longest_aspath' variable


# print and check the longest ASPATH
print "The longest ASPATH is: {}\n".format(longest_aspath)
check_longest_aspath(longest_aspath)

### 问题

#### Q6.
如果你在线搜索“AS number lookup”，会找到若干 AS 查询服务。查找最长路由中的这些 AS，找出它们的所属国家，并按顺序列出这些国家。

#### A6.
*TODO：在此填写你的答案。*


#### Q7.
为什么这么长的 BGP 路由会令网络运营者担忧？请给出至少 2 个原因。

#### A7.
*TODO：在此填写你的答案。*


## 提交

**在关闭 notebook 或选项卡之前，请务必使用菜单中的“保存并检查点（Save and Checkpoint）”。**